In [1]:

import google.colab
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
data_path ="/content/drive/MyDrive/Colab Notebooks/Data/job_intents.json"

In [3]:
# Step 0: Environment + imports + Colab-safe drive mount
import os
import json
import random
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD

In [7]:
# Colab-safe drive mount: attempt to mount if running on Colab, otherwise use cwd
try:
    import google.colab
    from google.colab import drive
    drive.mount('/content/drive')
    data_base_path = '/content/drive/MyDrive/Colab Notebooks/Data'
    print("Running on Colab. Using data_base_path =", data_base_path)
except Exception:
    data_base_path = os.getcwd()
    print("Not running on Colab. Using current working directory as data_base_path =", data_base_path)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Running on Colab. Using data_base_path = /content/drive/MyDrive/Colab Notebooks/Data


In [8]:
# Step 1: data path and NLTK downloads
data_path = os.path.join(data_base_path, 'job_intents.json')

if not os.path.exists(data_path):
    raise FileNotFoundError(f"Data file not found at: {data_path}\nPlease place job_intents.json in {data_base_path}")

# NLTK resources (download if needed)
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [9]:

# Step 2: load JSON intents
with open(data_path, 'r', encoding='utf-8') as f:
    intents = json.load(f)


In [10]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [12]:
lemmatizer = WordNetLemmatizer()
ignore_words = ['?', '!', '.', ',']

words = []       # all unique words (raw tokens, will lemmatize later)
classes = []     # unique intent tags
documents = []   # list of (tokenized_pattern, tag) tuples

for intent in intents.get('intents', []):
    tag = intent.get('tag')
    for pattern in intent.get('patterns', []):
        tokens = word_tokenize(pattern)
        words.extend(tokens)
        documents.append((tokens, tag))
    if tag not in classes:
        classes.append(tag)

# Lemmatize and lower-case words, remove ignored tokens, deduplicate & sort
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(set(words))
classes = sorted(set(classes))

print("Total unique words:", len(words))
print("Total classes:", len(classes))
print("Total documents:", len(documents))



Total unique words: 100
Total classes: 20
Total documents: 67


In [13]:
# Step 4: create bag-of-words vector function and build training set
def bag_of_words(tokenized_sentence, all_words):
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in tokenized_sentence]
    bag = [1 if w in sentence_words else 0 for w in all_words]
    return np.array(bag, dtype=int)

training = []
output_empty = [0] * len(classes)

for doc in documents:
    pattern_tokens = doc[0]
    tag = doc[1]
    bag = bag_of_words(pattern_tokens, words)
    output_row = output_empty.copy()
    output_row[classes.index(tag)] = 1
    training.append([bag, output_row])



In [14]:
# Step 5: shuffle and split into train_x, train_y
random.shuffle(training)

# --- FIX: use list comprehensions to extract columns (works independent of numpy shape) ---
train_x = [t[0] for t in training]
train_y = [t[1] for t in training]

train_x = np.array(train_x, dtype=float)
train_y = np.array(train_y, dtype=float)

print("train_x shape:", train_x.shape)
print("train_y shape:", train_y.shape)

VALIDATION_SET = (train_x, train_y)


train_x shape: (67, 100)
train_y shape: (67, 20)


In [15]:
# Step 6: Create a simple feed-forward network
input_size = train_x.shape[1]
output_size = train_y.shape[1]

model = Sequential()
model.add(Dense(128, input_shape=(input_size,), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(output_size, activation='softmax'))

model.summary()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │        12,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 20)             │         1,300 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 22,484 (87.83 KB)

 Trainable params: 22,484 (87.83 KB)

 Non-trainable params: 0 (0.00 B)

In [16]:
# Step 7: Compile the model
sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


In [17]:
# Step 8: Train
history = model.fit(
    train_x,
    train_y,
    epochs=250,
    batch_size=5,
    validation_split=0.2,
    verbose=1
)


Epoch 1/250
11/11 ━━━━━━━━━━━━━━━━━━━━ 6s 223ms/step - accuracy: 0.0424 - loss: 3.0261 - val_accuracy: 0.0000e+00 - val_loss: 3.0211
Epoch 2/250
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.0111 - loss: 3.0209 - val_accuracy: 0.0000e+00 - val_loss: 3.0718
Epoch 3/250
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.1666 - loss: 2.9134 - val_accuracy: 0.0714 - val_loss: 3.1193
Epoch 4/250
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.1603 - loss: 2.7835 - val_accuracy: 0.0714 - val_loss: 3.1777
Epoch 5/250
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.3019 - loss: 2.6639 - val_accuracy: 0.0714 - val_loss: 3.2333
Epoch 6/250
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.2500 - loss: 2.6604 - val_accuracy: 0.0714 - val_loss: 3.2782
Epoch 7/250
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.1965 - loss: 2.5632 - val_accuracy: 0.0714 - val_loss: 3.2981
Epoch 8/250
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.3511 - loss: 2.4572 - val_accuracy: 

In [18]:
model.save('ds_chatbot_model.h5')
pickle.dump({
    'words': words,
    'classes': classes,
    'history': history.history
}, open('ds_chatbot_data.pkl', 'wb'))

print("Model saved to 'ds_chatbot_model.h5' and data saved to 'chatbot_data.pkl'")


Model saved to 'ds_chatbot_model.h5' and data saved to 'chatbot_data.pkl'


In [19]:
import json
import pickle

# ---------------------------
# 1. Load job_intents.json
# ---------------------------
with open(data_path, "r", encoding="utf-8") as f:
    intents = json.load(f)

In [20]:

# ---------------------------
# 2. Extract unique classes (tags)
# ---------------------------
classes = []

for intent in intents.get("intents", []):
    tag = intent.get("tag")
    if tag and tag not in classes:
        classes.append(tag)

# Sort classes
classes = sorted(classes)

print("Total classes found:", len(classes))
print(classes)

Total classes found: 20
['career_after_graduation', 'career_confusion', 'career_gap', 'career_motivation', 'career_switch', 'confidence_building', 'data_analyst_roadmap', 'data_science_roadmap', 'first_job_fear', 'goodbye', 'greeting', 'internship_guidance', 'interview_preparation', 'job_vs_higher_studies', 'linkedin_profile', 'project_importance', 'resume_tips', 'salary_expectation', 'skills_for_freshers', 'soft_skills']


In [21]:
# ---------------------------
# 3. Save classes.pkl
# ---------------------------
with open("classes.pkl", "wb") as f:
    pickle.dump(classes, f)

print("\nclasses.pkl file successfully created!")



classes.pkl file successfully created!


In [22]:
import json
import pickle
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

# Download required NLTK models (first time only)
nltk.download('punkt')
nltk.download('wordnet')

# --------------------------------------
# 1. Load job_intents.json
# --------------------------------------
with open(data_path, "r", encoding="utf-8") as f:
    intents = json.load(f)

# --------------------------------------
# 2. Preprocessing setup
# --------------------------------------
lemmatizer = WordNetLemmatizer()
ignore_words = ['?', '!', '.', ',']

words = []        # vocabulary list
documents = []    # (pattern, tag) pairs

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [23]:
# 3. Tokenize, lemmatize & collect words
# --------------------------------------
for intent in intents["intents"]:
    for pattern in intent["patterns"]:
        tokens = word_tokenize(pattern)               # tokenize
        words.extend(tokens)                          # add tokens
        documents.append((tokens, intent["tag"]))     # store structure

# --------------------------------------
# 4. Clean words → lower, lemmatize, remove symbols
# --------------------------------------
words = [lemmatizer.lemmatize(w.lower())
         for w in words
         if w not in ignore_words]

# unique + sorted
words = sorted(list(set(words)))

print("Total unique words:", len(words))
print(words[:50])   # first 50 words preview

# --------------------------------------
# 5. Save words.pkl
# --------------------------------------
with open("words.pkl", "wb") as f:
    pickle.dump(words, f)

Total unique words: 100
['a', 'about', 'after', 'am', 'analysis', 'analyst', 'anyone', 'are', 'become', 'build', 'bye', 'career', 'change', 'choose', 'college', 'confidence', 'confused', 'crack', 'data', 'day', 'degree', 'demotivated', 'do', 'expect', 'expected', 'explain', 'fear', 'feel', 'find', 'first', 'for', 'fresher', 'gap', 'get', 'good', 'graduation', 'guidance', 'hello', 'hey', 'hi', 'higher', 'how', 'i', 'importance', 'important', 'improve', 'in', 'internship', 'interview', 'is']
